若有多个模块，或者部分模块可以人为分为一组，则可以使用`ModuleList`或`Sequential`，它们之间的区别是：`ModuleList`没有实现`forward`函数，也就是无法直接调用，`Sequential`可以。

### Sequential
#### 写法1

In [2]:
import torch
from torch import nn

In [5]:
net1 = nn.Sequential()
net1.add_module('conv', nn.Conv2d(3,3,3))
net1.add_module('batchnorm', nn.BatchNorm2d(3))
net1.add_module('activation_layer', nn.ReLU())

#### 写法2

In [6]:
net2 = nn.Sequential(
    nn.Conv2d(3,3,3),
    nn.BatchNorm2d(3),
    nn.ReLU()
)


#### 写法3

In [13]:
from collections import OrderedDict
net3 = nn.Sequential(OrderedDict([
    ('conv', nn.Conv2d(3,3,3)),
    ('batchnorm', nn.BatchNorm2d(3)),
    ('activation_layer', nn.ReLU()),
]))

#### 使用

In [21]:
print("net1:", net1)
print("net2:", net2)
print("net3:", net3)

net1: Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation_layer): ReLU()
)
net2: Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
net3: Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation_layer): ReLU()
)


In [19]:
# 课根据名字或序号取得子Module
net1.conv, net2[0], net3.conv

(Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)),
 Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)),
 Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)))

In [29]:
input = torch.rand(1,3,4,4)
output = net1(input)
output = net2(input)
output = net3(input)
output = net3.activation_layer(net1.batchnorm(net1.conv(input)))

### ModuleList

In [31]:
modellist = nn.ModuleList([
    nn.Linear(3,4),
    nn.ReLU(),
    nn.Linear(4,2)
])
input = torch.randn(2,3)
for model in modellist:
    input = model(input)

In [33]:
# error!
# 因为 modelist没有实现forward方法
output = modellist(input)

NotImplementedError: 

In [37]:
class MyMoudle(nn.Module):
    def __init__(self):
        super(MyMoudle, self).__init__()
        self.list = [nn.Linear(3,4), nn.ReLU()]
        self.module_list = nn.ModuleList([nn.Linear(3,4),nn.Linear(4,2)])
    def forward(self):
        pass
    
model = MyMoudle()
model

MyMoudle(
  (module_list): ModuleList(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): Linear(in_features=4, out_features=2, bias=True)
  )
)

In [38]:
for name,para in model.named_parameters():
    print(name,para)

module_list.0.weight Parameter containing:
tensor([[-0.0445,  0.4815, -0.5278],
        [ 0.3126,  0.1912, -0.2584],
        [ 0.4708,  0.2708,  0.3358],
        [ 0.2541, -0.3848, -0.1810]], requires_grad=True)
module_list.0.bias Parameter containing:
tensor([ 0.2951, -0.1484, -0.3788,  0.5285], requires_grad=True)
module_list.1.weight Parameter containing:
tensor([[-0.4703,  0.3544, -0.1640,  0.2284],
        [-0.3891,  0.4264, -0.3632, -0.1653]], requires_grad=True)
module_list.1.bias Parameter containing:
tensor([ 0.1788, -0.4328], requires_grad=True)


从上面结果可以看出，list中的子module不能被pytorch识别。